# License
Creative Commons Attribution-NonCommercial 4.0 International Public License

By exercising the Licensed Rights (defined below), You accept and agree to be bound by the terms and conditions of this Creative Commons Attribution-NonCommercial 4.0 International Public License ("Public License"). To the extent this Public License may be interpreted as a contract, You are granted the Licensed Rights in consideration of Your acceptance of these terms and conditions, and the Licensor grants You such rights in consideration of benefits the Licensor receives from making the Licensed Material available under these terms and conditions.

...

You are free to:
- Share — copy and redistribute the material in any medium or format
- Adapt — remix, transform, and build upon the material

The licensor cannot revoke these freedoms as long as you follow the license terms.

Under the following terms:
- Attribution — You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
- NonCommercial — You may not use the material for commercial purposes.

...

## Full License Text: https://creativecommons.org/licenses/by-nc/4.0/legalcode

In [4]:
# Install Tor and necessary packages
!apt-get install tor -y
!pip install stem requests[socks] pyngrok

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tor is already the newest version (0.4.6.10-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [9]:
import os
import time
import requests
from stem import Signal
from stem.control import Controller
from stem.process import launch_tor_with_config
from pyngrok import ngrok
import pandas as pd
import socket

# Settings
settings = {
    "timeout": 35,
    "control_domain": "google.com",
    "test_domain": "apple.com",
    "max_retries_per_country": 5,
    "ngrok_authtoken": "your_ngrok_authtoken",
    "country_codes": [
        'US', 'CA', 'DE', 'FR', 'GB', 'AU', 'JP', 'KR', 'RU', 'BR', 'ES', 'IT', 'CH', 'SE', 'FI', 'NO', 'DK',
        'BE', 'AT', 'PL', 'CZ', 'RO', 'UA', 'HU', 'GR', 'BG', 'TR', 'IL', 'ZA', 'SG', 'HK', 'MY', 'ID', 'PH', 'VN', 'TH'
    ]
}

# Ngrok authtoken
ngrok_authtoken = settings["ngrok_authtoken"]

# Authenticate Ngrok
!ngrok authtoken $ngrok_authtoken

# Function to terminate any existing Tor processes
def kill_existing_tor():
    os.system("pkill -f tor")

# Kill existing Tor processes
kill_existing_tor()

# Generate a hashed password for Tor
password = 'my_password'  # You can set your desired password here
hashed_password = os.popen(f'tor --hash-password {password}').read().strip().split('\n')[-1]
print(f"Hashed Password: {hashed_password}")

# Function to launch Tor with specific ExitNodes
def launch_tor_with_exit_nodes(exit_node_country):
    tor_config = {
        'ControlPort': '9051',
        'HashedControlPassword': hashed_password,
        'ExitNodes': f'{{{exit_node_country}}}',
        'StrictNodes': '1',
        'Log': 'NOTICE stdout'
    }
    return launch_tor_with_config(config=tor_config, timeout=300, init_msg_handler=lambda line: print(line) if 'Bootstrapped' in line else None)

# Function to resolve a domain name to an IP address
def resolve_domain_to_ip(domain):
    return socket.gethostbyname(domain)

# Function to get a new Tor identity
def renew_tor_ip(controller):
    controller.signal(Signal.NEWNYM)
    time.sleep(controller.get_newnym_wait())

# Function to check if a website is accessible
def check_website(url, retries=3):
    session = requests.session()
    session.proxies = {
        'http': 'socks5h://localhost:9050',
        'https': 'socks5h://localhost:9050'
    }
    for _ in range(retries):
        try:
            response = session.get(url, timeout=settings["timeout"])
            return response.status_code == 200
        except requests.RequestException as e:
            if "timed out" in str(e):
                continue
            return False
    return False

# The websites to check
websites = [f'http://{settings["control_domain"]}', f'http://{settings["test_domain"]}']

# Log the results
results = []

# Expose the control port via Ngrok
ngrok_tunnel = ngrok.connect(9051, "tcp")
print(f"Ngrok Tunnel: {ngrok_tunnel.public_url}")

# Resolve the ngrok host to an IP address
ngrok_host, ngrok_port = ngrok_tunnel.public_url[6:].split(":")
ngrok_ip = resolve_domain_to_ip(ngrok_host)
ngrok_port = int(ngrok_port)

# Iterate through the list of countries
for country_code in settings["country_codes"]:
    retry_count = 0
    success = False

    while retry_count < settings["max_retries_per_country"] and not success:
        try:
            # Launch Tor with the specified exit node country
            tor_process = launch_tor_with_exit_nodes(country_code)
            time.sleep(30)  # Wait for Tor to start

            # Connect to Tor controller using the Ngrok tunnel
            with Controller.from_port(address=ngrok_ip, port=ngrok_port) as controller:
                controller.authenticate(password=password)

                # Check control domain accessibility first
                control_domain_accessible = check_website(websites[0])

                if control_domain_accessible:
                    # Check test domain accessibility
                    test_domain_accessible = check_website(websites[1])

                    # Log results
                    results.append({
                        'Country': country_code,
                        'Control Domain Accessible': control_domain_accessible,
                        'Test Domain Accessible': test_domain_accessible
                    })
                    print(f"Country: {country_code}, {settings['control_domain']} Accessible: {control_domain_accessible}, {settings['test_domain']} Accessible: {test_domain_accessible}")
                    success = True
                else:
                    print(f"Country: {country_code}, {settings['control_domain']} not accessible. Retrying with a new endpoint.")
                    retry_count += 1

            # Terminate the current Tor process
            tor_process.terminate()
            time.sleep(5)  # Allow time for termination before restarting

        except Exception as e:
            error_message = str(e)
            if "reached a 300 second timeout without success" in error_message:
                print(f"Country: {country_code}, Error: Timeout occurred. Retrying with a new endpoint.")
            else:
                print(f"Country: {country_code}, Error: {error_message}")
                results.append({'Country': country_code, 'Control Domain Accessible': False, 'Test Domain Accessible': False, 'Error': error_message})
            retry_count += 1

# Print the results
df = pd.DataFrame(results)
df.to_csv('website_accessibility_results.csv', index=False)
df


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Hashed Password: 16:7E56FFDCAC812E8260B37F41B7531A2E53A03772BAD66FDE2F63046117
Ngrok Tunnel: tcp://0.tcp.ngrok.io:15826
Jul 23 09:40:26.000 [notice] Bootstrapped 0% (starting): Starting
Jul 23 09:40:26.000 [notice] Bootstrapped 5% (conn): Connecting to a relay
Jul 23 09:40:26.000 [notice] Bootstrapped 10% (conn_done): Connected to a relay
Jul 23 09:40:27.000 [notice] Bootstrapped 14% (handshake): Handshaking with a relay
Jul 23 09:40:27.000 [notice] Bootstrapped 15% (handshake_done): Handshake with a relay done
Jul 23 09:40:27.000 [notice] Bootstrapped 75% (enough_dirinfo): Loaded enough directory info to build circuits
Jul 23 09:40:27.000 [notice] Bootstrapped 85% (ap_conn_done): Connected to a relay to build circuits
Jul 23 09:40:28.000 [notice] Bootstrapped 89% (ap_handshake): Finishing handshake with a relay to build circuits
Jul 23 09:40:28.000 [notice] Bootstrapped 90% (ap_handshake_done): Handshake finished wit

DEBUG:stem:GETCONF __owningcontrollerprocess (runtime: 0.3486)
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content
INFO:stem:Error while receiving a control message (SocketClosed): empty socket content


Country: US, google.com Accessible: True, apple.com Accessible: True
Jul 23 09:41:11.000 [notice] Bootstrapped 0% (starting): Starting
Jul 23 09:41:11.000 [notice] Bootstrapped 5% (conn): Connecting to a relay
Jul 23 09:41:13.000 [notice] Bootstrapped 10% (conn_done): Connected to a relay
Jul 23 09:41:13.000 [notice] Bootstrapped 14% (handshake): Handshaking with a relay
Jul 23 09:41:13.000 [notice] Bootstrapped 15% (handshake_done): Handshake with a relay done
Jul 23 09:41:13.000 [notice] Bootstrapped 75% (enough_dirinfo): Loaded enough directory info to build circuits
Jul 23 09:41:13.000 [notice] Bootstrapped 90% (ap_handshake_done): Handshake finished with a relay to build circuits
Jul 23 09:41:13.000 [notice] Bootstrapped 95% (circuit_create): Establishing a Tor circuit
Jul 23 09:41:14.000 [notice] Bootstrapped 100% (done): Done


DEBUG:stem:GETCONF __owningcontrollerprocess (runtime: 0.3484)
INFO:stem:Error while receiving a control message (SocketClosed): received exception "read of closed file"


Country: CA, google.com not accessible. Retrying with a new endpoint.
Jul 23 09:41:53.000 [notice] Bootstrapped 0% (starting): Starting
Jul 23 09:41:54.000 [notice] Bootstrapped 5% (conn): Connecting to a relay
Jul 23 09:41:55.000 [notice] Bootstrapped 10% (conn_done): Connected to a relay
Jul 23 09:41:55.000 [notice] Bootstrapped 14% (handshake): Handshaking with a relay
Jul 23 09:41:56.000 [notice] Bootstrapped 15% (handshake_done): Handshake with a relay done
Jul 23 09:41:56.000 [notice] Bootstrapped 75% (enough_dirinfo): Loaded enough directory info to build circuits
Jul 23 09:41:56.000 [notice] Bootstrapped 90% (ap_handshake_done): Handshake finished with a relay to build circuits
Jul 23 09:41:56.000 [notice] Bootstrapped 95% (circuit_create): Establishing a Tor circuit
Jul 23 09:41:57.000 [notice] Bootstrapped 100% (done): Done


...

DEBUG:stem:GETCONF __owningcontrollerprocess (runtime: 0.3485)
INFO:stem:Error while receiving a control message (SocketClosed): received exception "read of closed file"


Country: VN, google.com Accessible: True, apple.com Accessible: True
Jul 23 11:14:02.000 [notice] Bootstrapped 0% (starting): Starting
Jul 23 11:14:02.000 [notice] Bootstrapped 5% (conn): Connecting to a relay
Jul 23 11:14:03.000 [notice] Bootstrapped 10% (conn_done): Connected to a relay
Jul 23 11:14:04.000 [notice] Bootstrapped 14% (handshake): Handshaking with a relay
Jul 23 11:14:04.000 [notice] Bootstrapped 15% (handshake_done): Handshake with a relay done
Jul 23 11:14:04.000 [notice] Bootstrapped 75% (enough_dirinfo): Loaded enough directory info to build circuits
Jul 23 11:14:04.000 [notice] Bootstrapped 90% (ap_handshake_done): Handshake finished with a relay to build circuits
Jul 23 11:14:04.000 [notice] Bootstrapped 95% (circuit_create): Establishing a Tor circuit
Jul 23 11:14:05.000 [notice] Bootstrapped 100% (done): Done


DEBUG:stem:GETCONF __owningcontrollerprocess (runtime: 0.3487)
INFO:stem:Error while receiving a control message (SocketClosed): received exception "read of closed file"


Country: TH, google.com Accessible: True, apple.com Accessible: True


,Country,Control Domain Accessible,Test Domain Accessible
0,US,True,True
1,CA,True,True
2,DE,True,True
3,FR,True,True
4,GB,True,True
5,AU,True,True
6,JP,True,True
7,KR,True,True
8,RU,True,True
9,BR,True,True
